In [ ]:
# This is a code for Chinese A-share Stock Trading Strategy with JoinQuant API based on factor model
from jqdata import *
import numpy as np
import pandas as pd
import datetime

In [ ]:
# Initialization
def initialize(context):
    # Set CSI300 as benchmark
    set_benchmark('000300.XSHG')
    set_option('use_real_price', True)
    log.info('Run the initial function for once')
    # log.set_level('order', 'error')

    ### Trading Settings
    # Transaction Fees：3/10000(commission) for buying，3/10000(commission) plus 1/1000(Stamp Duty); 5 rmb for minimal commission
    set_order_cost(OrderCost(close_tax=0.001, open_commission=0.0003, close_commission=0.0003, min_commission=5), type='stock')
    ## Strategy Settings
    set_params()
    set_variables()
    ## Run function
      # Before Market Opened
    run_daily(before_market_open, time='before_open', reference_security='000300.XSHG')
      # After Market Opened
    run_daily(market_open, time='every_bar', reference_security='000300.XSHG', )
      # After Market Closed
    run_daily(after_market_close, time='after_close', reference_security='000300.XSHG')

def set_params():
    g.days = 0
    # 25 for planA
    g.refresh_rate = 25
    # 100 for planB
    # g.refresh_rate = 100
    g.number_days = 20
    sys_short = sys_turtle("short", 20, 10, 4, 3, 0.8)
    sys_long  = sys_turtle("long",  55, 20, 4, 1, 0.2)
    g.systems = [sys_short, sys_long]
    g.pool_size = 10 # Pool Size
    g.WEIGHT_A = {
        'marc': 0.8,
        'val': 0.05, 
        'prof': 0.15, 
        'mana': 0.00
    }
    g.WEIGHT_VAL = {
        'EP': 0.3,
        'BP': 0.3,
        'SP': 0.2,
        'CFP': 0.2
    }
    g.WEIGHT_MARC = {
        'M_CAP': 1
    }
    g.WEIGHT_PROF = {
        'ROE': 0.3,
        'ROA': 0.3,
        'GSSR': 0.4 
    }
    g.WEIGHT_MANA = {
        'NIS': 0.3, # Issuance
        'MOM': 0.4         
    }
        
def set_variables():
    g.Ns = {}
    g.buy_stock_list = []
    
def load_raw_one(context):
    day = context.previous_date
    q = query(
        # Code
        valuation.code,
        # Market Capitalization
        valuation.market_cap,
        # Valuation
        valuation.pe_ratio,
        valuation.pb_ratio,
        valuation.ps_ratio,
        valuation.pcf_ratio,
        # Returns
        indicator.roe,
        indicator.roa,
        indicator.goods_sale_and_service_to_revenue,
        # Investment
        balance.total_assets,
        balance.fixed_assets,
        # Management
        valuation.circulating_cap,
        balance.total_owner_equities

    ).filter(
        valuation.pe_ratio <= 100,
        valuation.pe_ratio > 0
    ).order_by(
        valuation.market_cap.desc()
    )
    df_raw = get_fundamentals(q, date=day)
    return df_raw
    
def init_filter(context, df_raw):
    tmp_len = int(0.9*len(df_raw))
    df_raw = df_raw[:tmp_len]
    tmp_codes = np.array(df_raw['code'])
    tmp_arr = [] # codes for all ST stocks
    current_datas = get_current_data()
    for code in tmp_codes:
        current_data = current_datas[code]
        if current_data.is_st:
            tmp_arr.append(code)
    df_raw = df_raw[~df_raw['code'].isin(tmp_arr)]
    df_raw = df_raw.reset_index(drop=True)
    return df_raw
    
def ep_score(df, frac):
    """
    plus EP scores，return df
    """
    score = []
    df = df.sort_values(by='pe_ratio') # default: ascending by 'pe_ratio'
    df = df.reset_index(drop=True)
    df['score'][:frac] += 8*g.WEIGHT_VAL['EP']
    df['score'][frac:(2*frac)] += 4*g.WEIGHT_VAL['EP']
    df['score'][(2*frac):(3*frac)] += 2*g.WEIGHT_VAL['EP']
    df['score'][(3*frac):] += 0.5*g.WEIGHT_VAL['EP']
    return df
    
def bp_score(df, frac):
    """
    plus BP scores，return df
    """
    # pandas==0.24.3 does not support key function of sort_values
    # df = df.sort_values(by='pb_ratio', key=lambda col: (1/col), ascending=False)
    df['for_sort'] = 1/df['pb_ratio']
    df = df.sort_values(by='for_sort', ascending=False).drop(columns=['for_sort'])
    df = df.reset_index(drop=True)

    df['score'][:frac] += 8*g.WEIGHT_VAL['BP']
    df['score'][frac:(2*frac)] += 4*g.WEIGHT_VAL['BP']
    df['score'][(2*frac):(3*frac)] += 2*g.WEIGHT_VAL['BP']
    df['score'][(3*frac):] += 0.5*g.WEIGHT_VAL['BP']
    return df

def sp_score(df, frac):
    """
    plus SP scores，return df
    """
    # pandas==0.24.3 does not support key function of sort_values
    # df = df.sort_values(by='ps_ratio', key=lambda col: (1/col), ascending=False)
    df['for_sort'] = 1/df['ps_ratio']
    df = df.sort_values(by='for_sort', ascending=False).drop(columns=['for_sort'])
    df = df.reset_index(drop=True)
    
    df['score'][:frac] += 8*g.WEIGHT_VAL['SP']
    df['score'][frac:(2*frac)] += 4*g.WEIGHT_VAL['SP']
    df['score'][(2*frac):(3*frac)] += 2*g.WEIGHT_VAL['SP']
    df['score'][(3*frac):] += 0.5*g.WEIGHT_VAL['SP']
    return df
    
def cfp_score(df, frac):
    """
    plus CFP scores，return df
    """
    # pandas==0.24.3 does not support key function of sort_values
    # df = df.sort_values(by='pcf_ratio', key=lambda col: (1/col), ascending=False)
    df['for_sort'] = 1/df['pcf_ratio']
    df = df.sort_values(by='for_sort', ascending=False).drop(columns=['for_sort'])
    df = df.reset_index(drop=True)
    
    df['score'][:frac] += 8*g.WEIGHT_VAL['CFP']
    df['score'][frac:(2*frac)] += 4*g.WEIGHT_VAL['CFP']
    df['score'][(2*frac):(3*frac)] += 2*g.WEIGHT_VAL['CFP']
    df['score'][(3*frac):] += 0.5*g.WEIGHT_VAL['CFP']
    return df
    
# Valuation    
def val_score(context, df, weight):
    """
    plus valuation factor scores，return df
    """
    frac = int(len(df)/5)
    df = ep_score(df, frac)
    df = bp_score(df, frac)
    df = sp_score(df, frac)
    df = cfp_score(df, frac)
    df['score'] *= weight
    return df

def marc_score(context, df, weight):
    """
    plus mkt cap factor scores，return df
    """
    df = df.sort_values(by='market_cap')
    df = df.reset_index(drop=True)
    frac = int(len(df)/5)
    df['score'][:frac] += 8*g.WEIGHT_MARC['M_CAP']*weight
    df['score'][frac:(2*frac)] += 5*g.WEIGHT_MARC['M_CAP']*weight
    df['score'][(2*frac):(3*frac)] += 3*g.WEIGHT_MARC['M_CAP']*weight
    df['score'][(3*frac):] += 1*g.WEIGHT_MARC['M_CAP']*weight
    
    return df    

def roe_score(df, frac, weight):
    """
    plus ROE scores，return df
    """
    df = df.sort_values(by='roe', ascending=False)
    df = df.reset_index(drop=True)
    
    df['score'][:frac] += 8*g.WEIGHT_PROF['ROE']*weight
    df['score'][frac:(2*frac)] += 4*g.WEIGHT_PROF['ROE']*weight
    df['score'][(2*frac):(3*frac)] += 2*g.WEIGHT_PROF['ROE']*weight
    df['score'][(3*frac):] += 0.5*g.WEIGHT_PROF['ROE']*weight
    return df
    
def roa_score(df, frac, weight):
    """
    plus ROA scores，return df
    """
    df = df.sort_values(by='roa', ascending=False)
    df = df.reset_index(drop=True)
    
    df['score'][:frac] += 8*g.WEIGHT_PROF['ROA']*weight
    df['score'][frac:(2*frac)] += 4*g.WEIGHT_PROF['ROA']*weight
    df['score'][(2*frac):(3*frac)] += 2*g.WEIGHT_PROF['ROA']*weight
    df['score'][(3*frac):] += 0.5*g.WEIGHT_PROF['ROA']*weight
    return df
    
def gssr_score(df, frac, weight):
    """
    plus GSSR scores，return df
    for NaN的: mean subtracted by 1 unit
    """
    s = 0
    tmp_score = []
    nacnt = df['goods_sale_and_service_to_revenue'].isna().sum()
    df = df.sort_values(by='goods_sale_and_service_to_revenue', na_position='first')
    df = df.reset_index(drop=True)
    
    l = len(df) - nacnt
    item = df['goods_sale_and_service_to_revenue']
    for i in range(l):
        j = i + nacnt
        if item[j] > 110:
            s = 8*g.WEIGHT_PROF['GSSR']*weight
        elif item[j] > 100:
            s = 6*g.WEIGHT_PROF['GSSR']*weight
        elif item[j] > 80:
            s = 4*g.WEIGHT_PROF['GSSR']*weight
        else:
            s = 2*g.WEIGHT_PROF['GSSR']*weight
        #df['score'][i] += s
        tmp_score.append(s)
    df['score'][nacnt:] += tmp_score 
    mean_of_nnan = np.mean(tmp_score)
    score_of_nan = mean_of_nnan - 1*g.WEIGHT_PROF['GSSR']*weight
    df['score'][:nacnt] += score_of_nan
    return df

# Returns
def prof_qual_score(context, df, weight):
    """
    plus Return scores，return df
    """
    frac = int(len(df)/5)
    
    df = roe_score(df, frac, weight)
    
    df = roa_score(df, frac, weight)
    
    df = gssr_score(df, frac, weight)
    
    return df

def nis_score(df, sfrac, weight):
    """
    plus Net Issuance scores，return df
    """
    df = df.sort_values(by='circulating_cap')
    df = df.reset_index(drop=True)
    
    df['score'][:(3*sfrac)] += 8*g.WEIGHT_MANA['NIS']*weight
    df['score'][(3*sfrac):(5*sfrac)] += 5*g.WEIGHT_MANA['NIS']*weight
    df['score'][(5*sfrac):(8*sfrac)] += 3*g.WEIGHT_MANA['NIS']*weight
    df['score'][(8*sfrac):] += 1*g.WEIGHT_MANA['NIS']*weight
    
    return df
    
def mom_score(context, df, sfrac, weight):
    """
    plus MOM scores，return df
    """
    day = context.previous_date
    last = day - datetime.timedelta(days = g.refresh_rate)
    tmp_codes = df['code'][:]
    tmp_codes = list(tmp_codes)
    tmp_mom = []
    tmp_depre = []
    for code in tmp_codes:
        i = 0
        m = pd.DataFrame(data={'close':[np.nan]})
        m_now = pd.DataFrame(data={'close':[np.nan]})
        while m['close'][0] == np.nan and i <=3:
            m = get_price(code, count=1, end_date=(last - datetime.timedelta(days = i)), fields=['close'])
            i += 1
        while m_now['close'][0] == np.nan and i <=3:
            m_now = get_price(code, count=1, end_date=(day - datetime.timedelta(days = i)), fields=['close'])
            i += 1
        if m['close'][0] == np.nan or m_now['close'][0] == np.nan:
            tmp_depre.append(code)
        else:
            tmp_mom.append((m_now['close'][0] - m['close'][0])/m['close'][0])
    df = df[~df['code'].isin(tmp_depre)]
    df = df.reset_index(drop=True)
    df['mom'] = tmp_mom
    df = df.sort_values(by='mom', ascending=False)
    df = df.reset_index(drop=True)
    df['score'][:(2*sfrac)] += 8*g.WEIGHT_MANA['MOM']*weight
    df['score'][(2*sfrac):(4*sfrac)] += 4*g.WEIGHT_MANA['MOM']*weight
    df['score'][(4*sfrac):(6*sfrac)] += 2*g.WEIGHT_MANA['MOM']*weight
    df['score'][(6*sfrac):] += 0.5*g.WEIGHT_MANA['MOM']*weight
    
    return df

# Management
def mana_score(context, df, weight):
    """
    plus Management scores，return df
    """
    sfrac = int(len(df)/10)
    df = nis_score(df, sfrac, weight)
    
    df = mom_score(context, df, sfrac, weight)
    
    return df

def planA(context, df_raw):
    """
    Plan 1，return 50 codes in list
    """
    
    df_raw = load_raw_one(context)
    df_raw = init_filter(context, df_raw)
    l = len(df_raw)
    score = list(np.zeros(l))
    df_raw['score'] = score
    df = df_raw
    
    df = val_score(context, df, g.WEIGHT_A['val'])
    
    df = marc_score(context, df, g.WEIGHT_A['marc'])
    
    df = prof_qual_score(context, df, g.WEIGHT_A['prof'])
    
    df = mana_score(context, df, g.WEIGHT_A['mana'])
    
    df = df['code'][:g.pool_size]
    # df = df['code'][:-g.pool_size]
    df = list(df)
    return df

    
def before_market_open(context):
    if g.days % g.refresh_rate == 0:
        
        df_raw = pd.DataFrame()
        
        g.buy_stock_list = planA(context, df_raw)
        
        refresh_N()
    
    update_N()
    for sys in g.systems:
        sys.minute_count = 0
        sys.load_in_out_data()

def refresh_N():
    new_Ns = {}
    suspicious_too_high = []
    for sys in g.systems:
        for stock in sys.holding:
            if stock not in new_Ns:
                new_Ns[stock] = g.Ns[stock]
    for stock in g.buy_stock_list:
        if stock in g.Ns:
            new_Ns[stock] = g.Ns[stock]
        else:
            new_Ns[stock] = calc_N(stock)
            suspicious_too_high.append(stock)
    g.Ns = new_Ns
    for sys in g.systems:
        sys.not_able_in = set()
        for stock in suspicious_too_high:
            sys.not_able_in.add(stock)
    
def calc_N(stock):
    price = attribute_history(stock, g.number_days, '1d',('high','low','pre_close'))
    lst = []
    for i in range(0, g.number_days-1):
        h_l = price['high'][i]-price['low'][i]
        h_c = price['high'][i]-price['pre_close'][i]
        c_l = price['pre_close'][i]-price['low'][i]
        # Calculate True Range
        True_Range = max(h_l, h_c, c_l)
        lst.append(True_Range)
    return [np.mean(np.array(lst))]

def update_N():
    for stock in g.Ns:
        price = attribute_history(stock, 1, '1d',('high','low','pre_close'))
        h_l = price['high'][0]-price['low'][0]
        h_c = price['high'][0]-price['pre_close'][0]
        c_l = price['pre_close'][0]-price['low'][0]
        # Calculate the True Range
        True_Range = max(h_l, h_c, c_l)
        # Calculate the average of True_Range of last g.number_days（>20）days, i.e. the present value of N：
        current_N = (True_Range + (g.number_days-1)*(g.Ns[stock])[-1])/g.number_days
        (g.Ns[stock]).append(current_N)
    


def market_open(context):
    for sys in g.systems:
        sys.operate(context)

def after_market_close(context):
    g.days += 1

class sys_turtle:
    def __init__(self, name, in_date, out_date, unit_limit, holding_limit, ratio):
        self.name = name
        self.in_date = in_date
        self.out_date = out_date
        self.unit_limit = unit_limit
        self.holding_limit = holding_limit
        self.ratio = ratio
        
        self.holding = {}
        self.minute_count = 0
        self.day_in_data = {}
        self.day_out_data = {}
        self.not_able_in = set()
    
    def load_in_out_data(self):
        self.day_in_data = {}
        for stock in g.Ns:
            price = attribute_history(stock, self.in_date, '1d', ('close'))
            self.day_in_data[stock] = max(price['close'])
        
        self.day_out_data = {}
        for stock in g.Ns:
            price = attribute_history(stock, self.out_date, '1d', ('close'))
            self.day_out_data[stock] = min(price['close'])
        
    def operate(self, context):
        if self.minute_count % 20 == 0:
            #whether these stock is below blablabla
            if len(self.not_able_in) > 0:
                new_set = set()
                for stock in self.not_able_in:
                    current_price = get_bars(stock, count=1, unit='1m', fields=('close'), include_now=True, df=False)[0][0]
                    if current_price > self.day_in_data[stock]:
                        new_set.add(stock)
                self.not_able_in = new_set
            last_holding = {o: self.holding[o] for o in self.holding}
            for stock in last_holding:
                current_price = get_bars(stock, count=1, unit='1m', fields=('close'), include_now=True, df=False)[0][0]
                open_price = get_bars(stock, count=1, unit='1d', fields=('open'), include_now=True, df=False)[0][0]
                if (current_price >= 1.1*open_price or current_price <= 0.9*open_price):
                    continue
                self.stop_loss(stock, current_price, last_holding)
                self.market_out(stock, current_price, last_holding)
                
            per_value = context.portfolio.portfolio_value*self.ratio/self.holding_limit
            per_cash = context.portfolio.available_cash*self.ratio/self.holding_limit
            for stock in g.buy_stock_list:
                
                if stock in self.not_able_in:
                    log.info("enter! "+ stock)
                    continue
                current_price = get_bars(stock, count=1, unit='1m', fields=('close'), include_now=True, df=False)[0][0]
                open_price = get_bars(stock, count=1, unit='1d', fields=('open'), include_now=True, df=False)[0][0]
                if (current_price >= 1.095*open_price or current_price <= 0.905*open_price):
                    
                    continue
                ideal_unit = per_value * 0.01 / g.Ns[stock][-1] / 100
                if stock not in self.holding:
                    self.market_in(stock, current_price, per_cash, ideal_unit)
                else:
                    self.market_add(stock, current_price, ideal_unit, per_cash)
        self.minute_count += 1
            
        
    
    def market_in(self, stock, current_price, cash, ideal_unit):
        
        #price = attribute_history(stock, self.in_date, '1d', ('close'))
        if (current_price > self.day_in_data[stock]):
            actual_unit = ceil(ideal_unit)
            self.not_able_in.add(stock)
            if len(self.holding) == self.holding_limit:
                return
            if actual_unit*100*current_price>cash:
                actual_unit -= 1
            if actual_unit> 0 and actual_unit*100*current_price < cash:
                log.info(f"buy {self.name} {stock} {current_price} {actual_unit}")
                order(stock, actual_unit*100)
                self.holding[stock] = {'units': 1, 'price': current_price, 'numbers': actual_unit}
    
    def market_add(self, stock, current_price, ideal_unit, cash):
        if self.holding[stock]['units'] < self.unit_limit:
            if current_price > self.holding[stock]['price'] + g.Ns[stock][-1]/2:
                actual_unit = ceil(ideal_unit)
                if actual_unit*100*current_price>cash:
                    actual_unit -= 1
                if actual_unit> 0 and actual_unit*100*current_price < cash:
                    log.info(f"add {self.name} {stock} {current_price} {actual_unit}")
                    order(stock, actual_unit*100)
                    self.holding[stock]['units'] += 1
                    self.holding[stock]['price'] = current_price
                    self.holding[stock]['numbers'] += actual_unit
        pass
    
    def stop_loss(self, stock, current_price, lh):
        if current_price < lh[stock]['price'] - 2*g.Ns[stock][-1]:
            log.info(f"stop {self.name} {stock} {current_price}")
            order_target(stock, 0)
            self.not_able_in.add(stock)
            #order(stock, -self.holding[stock]['numbers']*100)
            for sys in g.systems:
                if stock in sys.holding:
                    sys.holding.pop(stock)
            
            
    def market_out(self, stock, current_price, lh):
        if stock not in self.holding:
            return
        #price = attribute_history(stock, self.out_date, '1d', ('close'))
        if current_price < self.day_out_data[stock]:
            log.info(f"leave {self.name} {stock} {current_price}")
            order_target(stock, 0)
            self.not_able_in.add(stock)
            #order(stock, -self.holding[stock]['numbers']*100)
            for sys in g.systems:  
                if stock in sys.holding:
                    sys.holding.pop(stock)
            
